# Семинар 8. Metric learning. AutoEncoder'ы. GAN'ы 🤔

## Немного терминов

**Metric learning** - это подход, в рамках которого мы строим функцию, которая оценивает "похожесть" двух объектов

- Почти любой человек сможет без труда отличить Монику и Рэйчел из "Друзей" между собой, даже если впервые видит их на фотографии. Хотим, чтобы то же самое можно было делать при помощи нейросети
- Основное отличие от классификации в том, что классы на `train` и `test` могут не совпадать
- Кроме того, для `metric learning` необязательно наличие классов и лейблов. Нам достаточно, например, знать только "похожесть" для всех пар

![](images/1.png)

*Так может выглядеть результат работы пайплайна по распознаванию лиц: детекция + кейпоинты + сеть на эмбеддинги*

[Видео с демонстрацией распознавания людей](https://www.youtube.com/watch?v=y-D1tReryGA)

**Autoencoder** - это нейронная сеть, которая учится эффективно сжимать или зашифровывать данные, получая их представление (вектор) в скрытом пространстве (latent space), и возвращать их в исходное пространство из скрытого в как можно более близком виде к исходному.

Для работы autoencoder'а не нужны лейблы → unsupervised learning

**GAN** - это нейросеть, состоящая из двух частей: генератора и дискриминатора. Генератор учится генерировать образцы, а дискриминатор учится отличать настоящие образцы от сгенерированных.

Как связаны эти понятия между собой?

$Metric\ learning →(Embedding)→ Autoencoder →(Latent\ space)→ GAN$


## Metric learning

Простой пример такого класса задач - **face recognition** (распознавание лиц)
- Кто из людей из тренировочных данных изображен на новом фото?
- На двух новых фото один и тот же человек или нет?
- Даны 3 фото нового человека. Он ли изображен на 4-том фото?

[Лекция по Metric Learning](https://www.youtube.com/watch?v=aU9yEwgrJ54)

Натренируем нейросеть на классификацию людей при помощи обычной свёрточной сети и посмотрим на её фичи. Если последний слой имеет, скажем, размер `[512, N]`, где `N` - это число людей в датасете, то на выходе из предпоследнего слоя для одной картинки получится вектор размера `[batch_size, N]`. Такие вектора называют **эмбеддингами**

Если мы возьмём все изображения, которые принадлежат одному и тому же человеку, и прогоним их через нашу сетку с оторванным слоем, то у нас получится много эмбеддингов. Эмбеддинги, принадлежащие одним и тем же людям, будут образовывать кластер.

Мы можем попытаться сравнивать центры этих кластеров между собой:

$$cos(y_1, y_2) = \frac{y_1 \times y_2}{||y_1||_2 ||y_2||_2}$$

Однако в общем случае ничто не гарантирует нам, что кластеры получатся плотными и узкими. Если нам на вход придут две фотографии новых людей, то из-за этого мы не сможем их разделить между собой. А вот пятилетний ребенок сможет :)



![](images/2.png)

*Эмбеддинги для разных фотографий одних и тех же людей образовывают кластеры в пространстве. Однако эти кластеры недостаточно плотные. Поэтому, если новый эмбеддинг образовывает точку между этими кластерами, мы не можем точно определить, какому классу соответствует эмбеддинг*

### Triplet loss

![](images/3.png)

*Схема для triplet loss'а*

Перед тем, как подать эмбеддинги на `triplet loss`, их необходимо нормализовать ($||f(x)||_2 = 1$)

Собираем из датасета положительные и отрицательные пары. Учим сеть, что расстояние между положительной парой должно быть маленьким, а расстояние между отрицательной - большим:

$$𝑳 = ∑_i^N max(||f(x_i^a) - f(x_i^p)||_2 - ||f(x_i^a) - f(x_i^n)||_2 + a, 0)$$

типичное значение $a=0.5$

1. Зачем ограничиваться 0 и использовать $a$? Если наши пары разделены сильнее, чем на $a$, то уже нет смысла обновлять
2. Как выбирать тройки?
  - Простые триплеты: $||f(x_i^a) - f(x_i^p)||_2 + a \lt ||f(x_i^a) - f(x_i^n)||_2$
  - Сложные триплеты: $||f(x_i^a) - f(x_i^p)||_2 + a \gt ||f(x_i^a) - f(x_i^n)||_2$
  - Полусложные триплеты: $||f(x_i^a) - f(x_i^p)||_2 \lt ||f(x_i^a) - f(x_i^n)||_2 \lt ||f(x_i^a) - f(x_i^p)||_2 + a$
  
  В начале учимся на полусложных и в конце на сложных (триплеты при этом будут становиться всё проще для сети), иначе может произойти коллапс обучения

Достоинства метода:
- Показывает неплохой результат, используют в SOTA'х
- Не обязательно должны быть классы, можно просто довольствоваться выбором позитивных и негативных примеров

Недостатки метода:
- Может быть непросто обучать
- Не учитывает дисперсию внутри класса: расталкиваем два кластера на расстояние $a$, при этом размер кластера может быть такого же порядка

### ArcFace loss

Пусть fc-слой $W$ для перевода из фичей в классы имеет размер `[512, 1000]`. Каждый из столбцов этой матрицы соответствует каждому из 1000 классов. Например, если мы обучаем сеть распознавать лица знаменитостей, то у нас может быть столбец, соответствующий Ким Кардашьян или Анджелине Джоли.

Формулу для кросс-энтропии можно переписать:

![](images/formula_0.png)

Если вектор $x$ соответствует фото Ким Кардашьян, то мы хотим, чтобы скалярное произведение $x$ с соответствующему этой знаменитости столбцу было максимальным.

Как можно ещё модифицировать такую функцию потерь?

- **Center loss** - добавляем к Softmax дополнительный член, который поджимает кластеры к центру:
![](images/formula_1.png)
- **SphereFace** - нормируем столбцы $W$, скалярное произведение расписываем через произведение скаляров и косинус между $x$ и $W$:
![](images/formula_2.png)
Тот угол, который соответствует нужному классу, штрафуем. Сети будет выгодно сделать его меньше, чтобы получать больший косинус и всё ещё правильно угадывать класс → она будет сжимать в более узкий кластер
- **ArcFace**:
![](images/formula_3.png)

Почему ArcFace лучше?

![](images/4.png)

*Граница для принятия решения о принадлежности к разным классам*

[Подробнее о функциях потерь для задачи face recognition](https://neptune.ai/blog/how-to-choose-loss-function-for-face-recognition)

## Autoencoders

Autoencoder состоит из encoder'а и decoder'а

![](images/5.png)

*Autoencoder, изображенный на схеме выше, позволяет сжать и получить обратно изображение*

Зачем вообще всё это нужно?

Интерес для нас представляет именно latent space. Оказывается, в нем зашифрована вообще вся информация о изображении. С latent space можно делать интересные операции. Например, взять две фотографии человека (улыбающегося и с нейтральным выражением лица) и при помощи операции между векторами в latent space найти вектор улыбки

- Можем использовать latent space, которое получили при помощи VAE, для генерации: выбираем случайную точку и, прогоняя через декодер, получаем изображение
- Можем подавать зашумленную картинку и восстанавливать менее шумную версию

![](images/6.png)

## GANs

- Состоит из двух сетей - генератора и дискриминатора. Генератор учится генерировать объекты и обманывать дискриминатор, который пытается понять, настоящий это объект или сгенерированный

Процедура обучения:

1. Сгенерируем случайный шум $z$
2. Отдадим этот шум на генератор и получим $g(z)$ - то, что генератор нагенерировал с этим входом
3. Обновляем веса дискриминатора (используем бинарную кросс-энтропию):
  $$𝓛 = -\frac{1}{m}\sum_m{ln(D(x) + ln(1 - D(G(x)))} $$
  $$x ⊂ real\ images ⇒  D(x) → 1$$

4. Сгенерируем случайный шум $z$
5. Отдаем на генератор и получим $g(z)$
6. Обновляем веса генератора (используем бинарную кросс-энтропию):
  $$𝓛 = -\frac{1}{m}\sum_m{ln(1 - D(G(x)))} $$

![](images/7.png)

Не обязательно стартовать с шума, можно с какой-то картинки

Примеры применения:

- Super resolution
- Image-to-image translation
- Photograph editing, etc.

![](images/8.png)

*Результат работы CycleGAN*